In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import pdb
from sklearn.manifold import TSNE
import sys
sys.path.append('/root/ShapeBind/ImageBind-LoRA/datasets')
from shapetalk_embedding import ShapetalkEmb_ClsDataset
import plotly.express as px
import numpy as np
import tqdm

dataset = ShapetalkEmb_ClsDataset()

bs = 512
dataloader = DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=0)


data, labels = next(iter(dataloader))
labels = np.asarray(labels)

with torch.no_grad():
    for index, (shape_emb, label) in enumerate(tqdm.tqdm(dataloader)):
        # label = list[batch길이]
        # shape_emb = tensor[batch, 1024]
        labels = np.append(labels, np.asarray(label))
        data = torch.cat((data, shape_emb), dim = 0)


/root/anaconda3/envs/imagebind_LoRA/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 72/72 [00:46<00:00,  1.54it/s]


In [2]:
print(data.shape)
print(labels.shape)


torch.Size([36903, 1024])
(36903,)


In [ ]:
num_items = 100

reduced_data = []
reduced_labels = []

class_count = [0 for _ in range(30)]
for idx in range(len(labels)):
    if class_count[labels[idx]] <= num_items:
        class_count[labels[idx]] += 1
        reduced_data.append(data[idx])
        reduced_labels.append(labels[idx])
    else:
        pass


In [ ]:
# t-SNE 적용
tsne = TSNE(n_components=3, random_state=42)
data_tsne = tsne.fit_transform(reduced_data.numpy())

# 시각화를 위한 DataFrame 생성
import pandas as pd
# df = pd.DataFrame(data_tsne, columns=[0, 1, 2])
df = pd.DataFrame(data_tsne, columns=[0, 1, 2])
df['label'] = reduced_labels

# Plotly를 사용한 시각화
fig = px.scatter_3d(df, x=0, y=1, z=2, labels={'color': 'label'})
fig.show()
